In [19]:
import itertools
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import sys
import copy
import random
import math
import pandas as pd
import numpy as np
import sklearn 
from sklearn.metrics.pairwise import haversine_distances
from math import radians

In [20]:
df = pd.read_csv("prices_en.csv")

In [21]:
df.head()

,date,product,market,size,price
0,2019-01-02,Aleika High Grade Flour,Bars,kg,NaN
1,2019-01-02,Aleika High Grade Flour,Bayanzurkh,kg,NaN
2,2019-01-02,Aleika High Grade Flour,Bumbugur,kg,NaN
3,2019-01-02,Aleika High Grade Flour,Emart,kg,NaN
4,2019-01-02,Aleika High Grade Flour,Kharkhorin,kg,NaN


In [22]:
df['product'].unique()

array(['Aleika High Grade Flour', 'Altan Taria Flour, First Grade',
       'Altan Taria Flour, High Grade',
       'Altan Taria Packaged Flour, 1st Grade',
       'Altan Taria Packaged Flour, 2nd Grade',
       'Altan Taria Packaged Flour, High Grade',
       'Altan Taria Packaged Flour, Top Grade', 'Apple, Fuji',
       'Atar Brand Bread', 'Beef Boneless', 'Beef w/Bones',
       'Beet, Mongolian', 'Brown Bread', 'Butter, Suu LTD',
       'Cabbage, Mongolian', 'Carrot, Mongolian', 'Egg, Russian',
       'Goat Meat w/Bones', 'Gree Tea, Georgia', 'Horse Meat w/Bones',
       'Milk, Bulk', 'Mutton Boneless', 'Mutton w/Bones', 'Onion, China',
       'Potato, Mongolian', 'Round Bread, Czech Style', 'Sugar',
       'Suu Brand Milk, 0.5l', 'Suu Brand Milk, 1l', 'Vegatable Oil',
       'White Rice', 'Yellow Rice', 'Yogurt',
       'Reserve meat, beef w/bones', 'Reserve meat, goat w/bones',
       'Reserve meat, mutton w/bones'], dtype=object)

In [23]:
print(len(df['product'].unique()))

36


In [24]:
df['market'].unique()

array(['Bars', 'Bayanzurkh', 'Bumbugur', 'Emart', 'Kharkhorin',
       'Khuchit Shonkhor', 'Max/M Mart', 'Mercury/Ikh Nayad', 'Nomin',
       'Orgil', 'Sansar', 'Saruul', 'Tavan Erdene'], dtype=object)

In [25]:
print(len(df['market'].unique()))

13


In [26]:
df = df.dropna()
df

,date,product,market,size,price
13,2019-01-02,"Altan Taria Flour, First Grade",Bars,kg,950.0
14,2019-01-02,"Altan Taria Flour, First Grade",Bayanzurkh,kg,1100.0
15,2019-01-02,"Altan Taria Flour, First Grade",Bumbugur,kg,1000.0
16,2019-01-02,"Altan Taria Flour, First Grade",Emart,kg,1080.0
17,2019-01-02,"Altan Taria Flour, First Grade",Kharkhorin,kg,1100.0
...,...,...,...,...,...
61927,2021-12-08,Yogurt,Nomin,liter,3099.0
61928,2021-12-08,Yogurt,Orgil,liter,3100.0
61929,2021-12-08,Yogurt,Sansar,liter,3100.0
61930,2021-12-08,Yogurt,Saruul,liter,2900.0


In [27]:
df = df.reset_index(drop=True)
df

,date,product,market,size,price
0,2019-01-02,"Altan Taria Flour, First Grade",Bars,kg,950.0
1,2019-01-02,"Altan Taria Flour, First Grade",Bayanzurkh,kg,1100.0
2,2019-01-02,"Altan Taria Flour, First Grade",Bumbugur,kg,1000.0
3,2019-01-02,"Altan Taria Flour, First Grade",Emart,kg,1080.0
4,2019-01-02,"Altan Taria Flour, First Grade",Kharkhorin,kg,1100.0
...,...,...,...,...,...
46700,2021-12-08,Yogurt,Nomin,liter,3099.0
46701,2021-12-08,Yogurt,Orgil,liter,3100.0
46702,2021-12-08,Yogurt,Sansar,liter,3100.0
46703,2021-12-08,Yogurt,Saruul,liter,2900.0


In [28]:
#Geo locations of the markets 
"""
Bars = [47.90896101359238, 106.89173405469197]
Bayanzurkh = [47.92213103358118, 106.98071503989783]
Bumbugur = [47.92009994509366, 106.90022693283476]
Emart = [47.90035180689655, 106.9288296144467]
Kharkhorin = [47.911709231673115, 106.83747109910689]
Khuchit Shonkhor = [47.9437799851189, 106.90213249370248]
Max/M Mart = [47.90546073214754, 106.89718521236082]
Mercury/Ikh Nayad = [47.907237722589414, 106.9286452395914]
Nomin = [47.90528789656089, 106.94281923697814]
Orgil = [47.89590227096271, 106.90806628299865]
Sansar = [47.913040829689905, 106.94364249921205]
Saruul = [47.909681238220735, 106.901913341121]
Tavan Erdene = [47.922234503547706, 106.88120267003602]
"""


'\nBars = [47.90896101359238, 106.89173405469197]\nBayanzurkh = [47.92213103358118, 106.98071503989783]\nBumbugur = [47.92009994509366, 106.90022693283476]\nEmart = [47.90035180689655, 106.9288296144467]\nKharkhorin = [47.911709231673115, 106.83747109910689]\nKhuchit Shonkhor = [47.9437799851189, 106.90213249370248]\nMax/M Mart = [47.90546073214754, 106.89718521236082]\nMercury/Ikh Nayad = [47.907237722589414, 106.9286452395914]\nNomin = [47.90528789656089, 106.94281923697814]\nOrgil = [47.89590227096271, 106.90806628299865]\nSansar = [47.913040829689905, 106.94364249921205]\nSaruul = [47.909681238220735, 106.901913341121]\nTavan Erdene = [47.922234503547706, 106.88120267003602]\n'

In [29]:
#https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.haversine_distances.html

#formula to create the distance matrix; units in km 